# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [27]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [28]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,calama,-22.4624,-68.9272,294.37,8,0,7.72,CL,2023-07-29
1,1,port-aux-francais,-49.3498,70.2200,273.35,91,100,4.64,TF,2023-07-29
2,2,thompson,55.7433,-97.8635,298.25,41,100,7.72,CA,2023-07-29
3,3,kingston,17.9712,-76.7928,305.05,70,20,10.29,JM,2023-07-29
4,4,jamestown,-15.9277,-5.7161,291.04,73,100,12.60,SH,2023-07-29


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [29]:
%%capture --no-display

# Configure the map plot
city_data_df.hvplot.points('Lng'
                            ,'Lat'
                            ,global_extent=True
                            ,frame_height=500
                            ,tiles=True
                            ,geo='True'
                            ,size='Humidity'
                            ,hover_cols = ['City']
                            )


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [30]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.dropna()
ideal_df = ideal_df.loc[(ideal_df['Humidity'] >= 40) 
                        & (ideal_df['Humidity'] <= 80)
                        & (ideal_df['Cloudiness'] <= 80)
                        & (ideal_df['Wind Speed'] <= 10)
                        & (ideal_df['Max Temp'] <= 290)]

# ideal_df = ideal_df.loc[ideal_df['Country'] == 'IT']
ideal_df = ideal_df.reset_index(drop=True)

ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,5,sergeyevka,53.8810,67.4159,289.76,79,21,2.19,KZ,2023-07-29
1,7,blackmans bay,-43.0047,147.3180,288.62,64,55,6.16,AU,2023-07-29
2,11,kapa'a,-33.9290,18.4174,284.91,74,75,2.57,ZA,2023-07-29
3,31,port alfred,-33.5982,26.8915,286.51,66,32,5.81,ZA,2023-07-29
4,59,noosa heads,-26.4001,153.0910,289.16,65,27,1.52,AU,2023-07-29


### Step 3: Create a new DataFrame called `hotel_df`.

In [31]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [32]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

hotel_df.head()

C:\Users\nmizz\AppData\Local\Temp\ipykernel_15592\181772109.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Country,Lat,Lng,Humidity,Hotel Name
0,sergeyevka,KZ,53.8810,67.4159,79,
1,blackmans bay,AU,-43.0047,147.3180,64,
2,kapa'a,ZA,-33.9290,18.4174,74,
3,port alfred,ZA,-33.5982,26.8915,66,
4,noosa heads,AU,-26.4001,153.0910,65,
5,hermanus,ZA,-34.4175,19.2361,74,
6,jwaneng,BW,-24.6004,24.7303,55,
7,iqaluit,CA,63.7494,-68.5219,76,
8,port lincoln,AU,-34.7212,135.8592,72,
9,carnarvon,AU,-24.8826,113.6576,72,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [35]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel'
    ,'apiKey':geoapify_key
    ,'limit': 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    latitude = row['Lat']
    longitude = row['Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search


c:\Users\nmizz\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


sergeyevka - nearest hotel: Гостиница "Шанырак"
blackmans bay - nearest hotel: Villa Howden
kapa'a - nearest hotel: The Capital 15 On Orange
port alfred - nearest hotel: Ferryman
noosa heads - nearest hotel: RACV Noosa Resort
hermanus - nearest hotel: Aloe guest house
jwaneng - nearest hotel: Mokala Lodge & Teemane Casino
iqaluit - nearest hotel: Capital Suites Hotel
port lincoln - nearest hotel: Grand Tasman
carnarvon - nearest hotel: No hotel found
usinsk - nearest hotel: Виктория
dunedin - nearest hotel: Scenic Hotel Dunedin City
labytnangi - nearest hotel: Семь лиственниц
chivhu - nearest hotel: Vic's Travern
chimoio - nearest hotel: Hotel Amirana
anadyr - nearest hotel: Гостевой дом
bindura - nearest hotel: Mayfair
copiapo - nearest hotel: Hotel Il Riposo
isoka - nearest hotel: No hotel found
greymouth - nearest hotel: Royal Hotel
taupo - nearest hotel: Taupo Ikanui Timeshare Resort
tambovka - nearest hotel: No hotel found
lichinga - nearest hotel: Girassol 
bendigo - nearest hote

,City,Country,Lat,Lng,Humidity,Hotel Name
0,sergeyevka,KZ,53.8810,67.4159,79,"Гостиница ""Шанырак"""
1,blackmans bay,AU,-43.0047,147.3180,64,Villa Howden
2,kapa'a,ZA,-33.9290,18.4174,74,The Capital 15 On Orange
3,port alfred,ZA,-33.5982,26.8915,66,Ferryman
4,noosa heads,AU,-26.4001,153.0910,65,RACV Noosa Resort
5,hermanus,ZA,-34.4175,19.2361,74,Aloe guest house
6,jwaneng,BW,-24.6004,24.7303,55,Mokala Lodge & Teemane Casino
7,iqaluit,CA,63.7494,-68.5219,76,Capital Suites Hotel
8,port lincoln,AU,-34.7212,135.8592,72,Grand Tasman
9,carnarvon,AU,-24.8826,113.6576,72,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [36]:
%%capture --no-display

# Configure the map plot
hotel_df.hvplot.points('Lng'
                            ,'Lat'
                            ,global_extent=True
                            ,frame_height=500
                            ,tiles=True
                            ,geo='True'
                            ,hover_cols = ['City', 'Country', 'Hotel Name']
                            )

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)